In [1]:
!pip install -q openai --upgrade
!pip install -q langchain
!pip install -q unstructured
!pip install -q sentence_transformers
!pip install -q chromadb
!pip install -q tiktoken
!pip install langchain_community -qq

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-bigquery 2.34.4 requires packaging<22.0dev,>=14.3, but you have packaging 23.2 which is incompatible.
jupyterlab 4.0.11 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
jupyterlab-lsp 5.0.2 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
libpysal 4.9.2 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.
momepy 0.7.0 requires shapely>=2, but you have shapely 1.8.5.post1 which is incompatible.
osmnx 1.8.1 requires shapely>=2.0, but you have shapely 1.8.5.post1 which is incompatible.
spopt 0.6.0 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source

In [2]:
import pandas as pd

from langchain.llms import OpenAI
from langchain.embeddings import OpenAIEmbeddings

from langchain.text_splitter import  RecursiveCharacterTextSplitter
from langchain.document_loaders import DirectoryLoader
from langchain_community.document_loaders import UnstructuredExcelLoader

from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma

from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.chains.question_answering import load_qa_chain

OPENAI_API_KEY = "<openai-api-key>"


In [3]:
import re
import glob

books_path = "/kaggle/input/malawi-data/MW_TGBookletsExcel/MWTGBookletsExcel/*.xlsx"
booklets = glob.glob(books_path)

def remove_unwanted(text):
    pattern = re.compile(r'\\ufdd0|[\U0001F600-\U0001F64F\U0001F300-\U0001F5FF\U0001F680-\U0001F6FF\U0001F700-\U0001F77F\U0001F780-\U0001F7FF\U0001F800-\U0001F8FF\U0001F900-\U0001F9FF\U0001FA00-\U0001FA6F\U0001FA70-\U0001FAFF\U00002702-\U000027B0\U000024C2-\U0001F251]+', flags=re.UNICODE)
    clean_text = pattern.sub('', text)
    
    return clean_text


In [4]:
for booklet in booklets:
    print(booklet)
    df= pd.read_excel(booklet, names=['row', 'text'])
    df['text'] = df['text'].apply(lambda x: remove_unwanted(str(x)))
    df.to_excel(booklet.split("/")[-1], index=False)

/kaggle/input/malawi-data/MW_TGBookletsExcel/MWTGBookletsExcel/TG Booklet 2 Sections 1,2,3_final_04112021.xlsx
/kaggle/input/malawi-data/MW_TGBookletsExcel/MWTGBookletsExcel/TG Booklet 1 Introduction Module Booklet 1TG_final_04112021.xlsx
/kaggle/input/malawi-data/MW_TGBookletsExcel/MWTGBookletsExcel/TG Booklet 3 Section 4,5,6 ,7_final_04112021.xlsx
/kaggle/input/malawi-data/MW_TGBookletsExcel/MWTGBookletsExcel/TG Booklet 5 Section 10_final_04112021.xlsx
/kaggle/input/malawi-data/MW_TGBookletsExcel/MWTGBookletsExcel/TG Booklet 6_Section 11_final_04112021.xlsx
/kaggle/input/malawi-data/MW_TGBookletsExcel/MWTGBookletsExcel/TG Booklet 4 Sections 8, 9_final_04112021.xlsx


In [5]:
cleaned_booklets = glob.glob("/kaggle/working/*.xlsx")
loaders = [UnstructuredExcelLoader(f"{booklet}", engine="") for booklet in cleaned_booklets]
docs = []
for loader in loaders:
    docs.extend(loader.load())

In [6]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,
                                     chunk_overlap=20)
chunked_documents = text_splitter.split_documents(docs)


In [7]:
llm = OpenAI(openai_api_key=OPENAI_API_KEY, model="gpt-3.5-turbo-instruct", temperature=0)
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [8]:
vectordb = Chroma.from_documents(documents=chunked_documents, embedding=embeddings, persist_directory="chroma_db")

In [9]:
retriever = vectordb.as_retriever(k=3)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    verbose=True,
)


In [10]:
import time
def test_rag(qa, query):
    #print(f"Query: {query}\n")
    time_1 = time.time()
    result = qa.run(query)
    time_2 = time.time()
    time_taken = round(time_2-time_1, 3)

    return result,time_taken

In [11]:
test = pd.read_csv("/kaggle/input/malawi-data/Test.csv")

In [12]:
from tqdm import tqdm

results = []
sources = []
for question in tqdm(test["Question Text"]):
    
    try:
        result,_ = test_rag(qa, question)
        docs = vectordb.similarity_search(result)
        source = docs[0].metadata['source'].split("/")[-1]

        results.append(result)
        sources.append(source)
    except:

        results.append("Error")
        sources.append("Error")



/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...
